In [1]:
from readdata import read_images
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, cdist

In [2]:
testing_data = read_images('../Yale_Face_Database/Testing/')
training_data = read_images('../Yale_Face_Database/Training/')

In [192]:
def get_A(data):
    data_ = np.array(data)
    mean = np.mean(data_,axis=1)
    A = np.array([i-j for (i,j) in zip(data_, mean)])
    A /= np.sqrt(np.size(data_,axis=1)-1)
    return A.T
A = get_A(training_data)
print(A.shape)

(45045, 135)


In [194]:
tt = np.array([[1,4,6,5,3,2,4],[5,4,5,4,3,3,2]])
cv = np.cov(tt)
kk = get_A(tt).T @ get_A(tt)
print(cv)
print(kk)

[[2.95238095 0.19047619]
 [0.19047619 1.23809524]]
[[2.95238095 0.19047619]
 [0.19047619 1.23809524]]


In [11]:
C = A @ A.T
vals , vecs = np.linalg.eig(C)

# sort
vecs = vecs[:,np.argsort(vals)]
vals = vals[np.argsort(vals)]

vecs = np.flip(vecs,axis=0)
vals = np.flip(vals)

In [150]:
C.shape

(135, 135)

In [171]:
print(C.shape)
print(projected.shape)
print(vecs.shape)
y = A.T @ vecs[0]
print(y.shape)


(135, 135)
(135, 25)
(135, 135)
(45045,)


In [172]:
# at least generate face projection
# this projected is with size 135 x 25
    # VoA = training_data @ A.T
    # projected = VoA @ vecs[:25].T

# now im trying projection
# C : 135 x 135
# training : 135 x 45045
    # projected = C @ vecs[:25].T

# new projection policy, project to only the eigenvector with the largest corresponding eigenvalue
def get_projected(xs,y):
    # projected x onto y
    # xs contains serveral x, projected x onto y, and return ys
    xs_ = np.array(xs)
    y_norm = np.dot(y,y)
    for x in xs_:
        x = y * np.dot(x,y) / y_norm
    return xs_

v = A.T @ vecs[0]
train_projected = get_projected(training_data, v)

In [198]:
print(A.shape)

(45045, 135)


In [202]:
print(training_data.shape)

(135, 45045)


In [209]:
# Adjust the training_data to fit the projection input
VoA = training_data @ A
projected = VoA @ vecs[:25].T
X_pca = training_data.T.dot(projected)
print(X_pca.shape)
eigenfaces = np.swapaxes(X_pca,0,1)
print(eigenfaces.shape)
for i in range(10):
    plt.imsave("../output/reconstruction/LDA/reconstruction"+str(i)+".png", eigenfaces[i].reshape(231,195), cmap=plt.cm.gray)
# plt.imshow(eigenfaces[24].reshape((231,195)),cmap=plt.cm.gray)

(45045, 25)
(25, 45045)


In [174]:
# original test_projected
    # test_projected = testing_data @ A.T @ vecs[:25].T
    # print(test_projected.shape)

# new test_projected
    # print(np.cov(testing_data).shape)
    # # test_projected = np.cov(testing_data) @ vecs[:25].T
    
# new projection method, project only onto the first eigenvector v
test_projected = get_projected(testing_data, v)
print(test_projected.shape)

(30, 45045)


In [176]:
aa = np.array([[4,3,2,1],[1,2,3,4]])
bb = np.array([[3,4,5,1],[3,4,1,2],[3,4,1,2]])
cdist(aa,bb)

array([[3.31662479, 2.        , 2.        ],
       [4.58257569, 4.        , 4.        ]])

In [179]:
dist_testXtrain = cdist(test_projected, train_projected)
print(dist_testXtrain.shape)

(30, 135)


In [48]:
aa = np.array([[4,3,2,1],[1,2,3,4]])
bb = np.array([[3,4,5,1],[3,4,1,2],[3,4,1,2]])
cc = []
for i in aa:
    cc.append(i)
for i in bb:
    cc.append(i)
cc = np.array(cc)
print(cc)

[[4 3 2 1]
 [1 2 3 4]
 [3 4 5 1]
 [3 4 1 2]
 [3 4 1 2]]


In [182]:
def make_symmetric_matrix(lower_list, width):
    n = width
    M = np.zeros((n,n)) # Initialize nxn matrix
    triu = np.triu_indices(n, 1) # Find upper right indices of a triangular nxn matrix
    tril = np.tril_indices(n, -1) # Find lower left indices of a triangular nxn matrix
    M[triu] = lower_list # Assign list values to upper right matrix
    M[tril] = M.T[tril] # Make the matrix symmetric
    return M

def k_nearest_neighbor(dists, k):
    # dists: matrix indicate distance between every points pair
    # the first 135 rows are labeled training data points and their distance toward every other points
    # we only utilize (training_points, testing_points) pair
    label = []
    for i in range(1,16):
        for j in range(9):
            label.append(i)
    label = np.array(label)
    test_label = []
    for test_points in dists:
        idx = np.argpartition(test_points, k)
        test_label.append( np.argmax( np.bincount( label[idx[:k]] )))
    return test_label

In [185]:
test_label = k_nearest_neighbor(dist_testXtrain, 10)

In [186]:
print(test_label)

[10, 7, 3, 5, 4, 11, 1, 9, 9, 8, 4, 4, 1, 8, 8, 10, 15, 4, 10, 13, 7, 3, 10, 15, 7, 8, 2, 15, 6, 11]


In [96]:
tt = np.arange(20).reshape(4,5)
print(tt)
print(tt[np.ix_([2,3],[0,1,2])])
print(tt[np.ix_(np.arange(2,4),np.arange(0,3))])

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]
[[10 11 12]
 [15 16 17]]
[[10 11 12]
 [15 16 17]]


In [76]:
ttt = np.array([1,2,3,4,5])
for i in ttt[3:]:
    print(i)

4
5


In [61]:
label = []
for i in range(1,16):
    for j in range(9):
        label.append(i)


In [73]:
label = np.array(label)

counts = np.bincount(label[[3,9,7,10,11,54,53,54,56,53,54,53,53]])
print(np.argmax(counts))


6


In [58]:
AA = np.array([1, 7,5,3,2,4])
k = 3

idx = np.argpartition(AA, k)
print(idx)
print(AA[idx[:k]])

[3 0 4 5 1 2]
[3 1 2]


In [31]:

tmp = pdist(projected, 'sqeuclidean')
tmp1 = make_symmetric_matrix(tmp,135)
print(tmp1.shape)

(135, 135)


In [ ]:
print(projected.shape)

In [ ]:
print((A.T).shape)
print(AT_v_s[0].shape)

In [ ]:

v = np.array([np.linalg.solve(A.T, AT_v) for AT_v in AT_v_s])



In [ ]:
tmpary = np.array([[1,2,3],[4,5,6]])
ary = np.array([i for i in tmpary])
print(ary)

In [ ]:
print(len(vecs[0]))

In [ ]:
print(At_v.shape)

In [ ]:
print(C.shape)

In [ ]:
C @ At_v